# Removing missing values from a dataset

This notebook shows how to use tools from `Pandas` to eliminate missing data from a data frame.

In [2]:
import numpy as np # Basic numerics
import pandas as pd # Data handling
import matplotlib.pyplot as plt # Graphics

## An example, the *Palmer penguins* dataset

By way of illustration, I'll work with a dataset collected and made available by [Dr. Kristen
Gorman](https://www.uaf.edu/cfos/people/faculty/detail/kristen-gorman.php)
and the [Palmer Station, Antarctica LTER](https://lternet.edu/site/palmer-antarctica-lter/), a
member of the [Long Term Ecological Research
Network](https://lternet.edu/). It describes measurements on 344 penguins from 3 species and here we'll work with a slightly reduced version that, nevertheless, has some missing data indicated by `NaN`'s. If you're interested, you can read more about these data at [Dr. Allison Horst's GitHub page](https://github.com/allisonhorst/palmerpenguins).

In [3]:
# Set the URL
penguinDataURL = "https://raw.githubusercontent.com/JohnMount/Penguins/main/penguins.csv"

# Fetch the data and take a first look
orig_penguin_df = pd.read_csv(penguinDataURL)
orig_penguin_df.head(10)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,female
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,male
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,NaN
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN


The next bit of code produces summary stats for the numeric columns and you can see from the line of counts (which exclude `NaN`'s), that there are a couple of missing values in each column.

In [4]:
orig_penguin_df.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


## Dropping rows that have missing data

Here we use built-in features of `Pandas` to get rid of rows that have missing data. We'll do this in two ways: first with a single line of code, then, somewhat more laboriously, with Boolean-indexing, an approach that allows us to select only those rows where something is `True`. I've included the latter as it generalises to other cases where one might want to select a subset of the data.

### One-line version with `dropna()`

This is, by far, the simplest, most `Pandas`-idiomatic approach. I learned it from Nasir Saeed, a student who did this MSc in 2020-21.

In [5]:
# Drop rows with missing data and display the result
penguin_df = orig_penguin_df.dropna()
penguin_df.head(10)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,female
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,male
12,Adelie,Torgersen,41.1,17.6,182.0,3200.0,female
13,Adelie,Torgersen,38.6,21.2,191.0,3800.0,male
14,Adelie,Torgersen,34.6,21.1,198.0,4400.0,male


In [6]:
# See how many rows remain
print( "Started with " + str(len(orig_penguin_df)) + 
      " rows of data and finished with " + str(len(penguin_df)) +
      " NaN-free ones."
)

Started with 344 rows of data and finished with 333 NaN-free ones.


### Using Boolean indexing

This approach is a bit more work, but is more powerful and generalisable.

#### Which entries in the dataframe are missing?

We begin by making a table that has the same shape as the original data frame, but whose entries are either `True` or `False` according to whether the data is present (that is, non-`NaN`) or not.

In [7]:
# Get a dataframe of the same shape, but with Boolean entries
# that answer the question "Is this item of data is present?"
dataPresent_df = orig_penguin_df.notna()
dataPresent_df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True
3,True,True,False,False,False,False,False
4,True,True,True,True,True,True,True


#### Which rows have complete data?

In this step we work through the table built above row-by-row, producing a list-like object with one entry per row of the original dataframe. The $j$-th entry in this list is a Boolean variable that answers the question: *Is all the data in the $j$-th row is present?*"

In [8]:
# Get a boolean array with one entry per row of the
# origial data frame and is True if the corresponding 
# row has no missing data.
wholeRowPresent = dataPresent_df.all( axis=1 )
wholeRowPresent[0:10] # Look at the first few entries

0     True
1     True
2     True
3    False
4     True
5     True
6     True
7     True
8    False
9    False
dtype: bool

#### Selecting the rows with no missing data.

Finally, we use the Boolean vector constructed above to pick out the rows we want from the orignal data frame

In [9]:
# Get that subset of the original data frame whose rows have no missing values
penguin_df2 = orig_penguin_df.loc[wholeRowPresent]
penguin_df2.head(10)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,female
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,male
12,Adelie,Torgersen,41.1,17.6,182.0,3200.0,female
13,Adelie,Torgersen,38.6,21.2,191.0,3800.0,male
14,Adelie,Torgersen,34.6,21.1,198.0,4400.0,male


In [10]:
# Again, see how many rows remain
print( "Started with " + str(len(orig_penguin_df)) + 
      " rows of data and finished with " + str(len(penguin_df2)) +
      " NaN-free ones."
)

Started with 344 rows of data and finished with 333 NaN-free ones.


In [11]:
# Finally, check that the two approaches agree
if penguin_df.equals( penguin_df2):
    print( "The two dataframes are the same." )
else:
    print( "The two dataframes aren't the same?!?" )

The two dataframes are the same.
